# Ask Docs Agent

**Note**: This notebook is not free to run, you will need to create ~20K OpenAI `text-embedding-ada-002` embedding which do cost money. The Pinecone index can be run within the free tier.

In [ ]:
!pip install -qU datasets pod-gpt pinecone-client[grpc] langchain openai tqdm PyPDF2 tiktoken matplotlib seaborn 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
directory_path = '/content/drive'
directory_files = os.listdir(directory_path)

print(directory_files)

['.file-revisions-by-id', 'MyDrive', '.shortcut-targets-by-id', '.Trash-0']


Set api keys, etc:

In [ ]:
OPENAI_API_KEY = ""  # platform.openai.com
PINECONE_API_KEY = ""  # app.pinecone.io
PINECONE_ENV = "us-east4-gcp"

In [ ]:
import PyPDF2

books = ['Postnihilism v12.pdf',
         'Natural Language Cognitive Architecture v7.pdf',
         'Benevolent By Design v8.pdf',
         'Symphony of Thought v3.pdf']

texts = []

for book in books:

  # open PDF file
  with open('/content/drive/MyDrive/Chatbot/' + book, 'rb') as file:
      # create a PDF reader object
      reader = PyPDF2.PdfReader(file)

      # get the number of pages in the PDF file
      num_pages = len(reader.pages)

      book_text = ''

      # loop through each page and extract the text
      for i in range(num_pages):
          # get the current page object
          page = reader.pages[i]

          # extract the text from the page
          book_text += page.extract_text()

      texts.append(book_text)


Set up tokenizer

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [ ]:
tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [ ]:
token_counts = [tiktoken_len(text) for text in texts]

In [ ]:
print(f"""Min: {min(token_counts)}
Avg: {int(sum(token_counts) / len(token_counts))}
Max: {max(token_counts)}""")

Min: 23843
Avg: 59115
Max: 73633


Calculate the total price to make embeddings for this text.

In [ ]:
price = 0.0004 # price per 1000 tokens for the ada model
print('$ ' + str((price*int(sum(token_counts)/1000))))

$ 0.0944


Very Afordable

Now lets use a chunker to break the text into usable sizes for the vector database

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,  # number of tokens overlap between chunks
    length_function=tiktoken_len,
    separators=['\n\n', '\n', ' ', '']
)

In [ ]:
chunks = text_splitter.split_text(texts[3])
len(chunks)

195

Process the data into chucks

In [ ]:
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for idx, record in enumerate(tqdm(texts)):
    split_text = text_splitter.split_text(record)
    chunks.extend([{
        'id': str(uuid4()),
        'text': split_text[i],
        'chunk': i
    } for i in range(len(split_text))])

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
len(chunks)

665

Initialize the embedding Model

In [ ]:
import openai

# initialize openai API key
openai.api_key = OPENAI_API_KEY  #platform.openai.com

embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)

In the response res we will find a JSON-like object containing our new embeddings within the 'data' field.

In [ ]:
res.keys()
     

dict_keys(['object', 'data', 'model', 'usage'])

Inside 'data' we will find two records, one for each of the two sentences we just embedded. Each vector embedding contains 1536 dimensions (the output dimensionality of the text-embedding-ada-002 model.

In [ ]:
len(res['data'])

2

In [ ]:
len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])

(1536, 1536)

We will apply this same embedding logic to the books that have been chunked. But before doing so we must create a place to store the embeddings.

# Initializing the Index

Now we need a place to store these embeddings and enable a efficient vector search through them all. To do that we use Pinecone, we can get a free API key and enter it below where we will initialize our connection to Pinecone and create a new index.

In [ ]:
import pinecone

index_name = 'ask-ben'

# initialize connection to pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # app.pinecone.io (console)
    environment=PINECONE_ENV  # next to API key in console
)

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='dotproduct'
    )
# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

We can see the index is currently empty with a total_vector_count of 0. We can begin populating it with OpenAI text-embedding-ada-002 built embeddings like so:

In [ ]:
from tqdm.auto import tqdm
import datetime
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts, engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 665}},
 'total_vector_count': 665}

Now we've added all of our books to the index. With that we can move on to retrieval and then answer generation using GPT-3.5

# Retrieval

To search through our documents we first need to create a query vector xq. Using xq we will retrieve the most relevant chunks from the books, like so:

In [ ]:
query = "What is the most likely outcome with the invention of AGI"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=5, include_metadata=True)

In [ ]:
res

{'matches': [{'id': 'c3362826-44a1-4d8a-98a2-52181c66f103',
              'metadata': {'chunk': 153.0,
                           'text': 'Many people I ’ve talked to do not want a '
                                   'super powerful AGI to exist ever. \n'
                                   'Under no circumstances do they want Skynet '
                                   'to exist, even if it is benevolent by \n'
                                   'design. While I understand their '
                                   'sentiment, I think we must  accept the \n'
                                   'possibility that an arms race between nati '
                                   'ons will necessitate the creation of ever '
                                   '-\n'
                                   'more -powerful AGI systems. Similarly, we '
                                   'would rather live in a world without \n'
                                   'nuclear weapons or other WMD, but they 

With Retrieval done we can now work on giving this text to gpt-3.5 to produce answers.

# Retrieval Augmented Generation

GPT-3.5 is currently accessed via the ChatCompletions endpoint of OpenAI. To add the information we retrieved into the model, we need to pass it into our user prompts alongside our original query. We can do that like so:

In [ ]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

In [ ]:
print(augmented_query)

Many people I ’ve talked to do not want a super powerful AGI to exist ever. 
Under no circumstances do they want Skynet to exist, even if it is benevolent by 
design. While I understand their sentiment, I think we must  accept the 
possibility that an arms race between nati ons will necessitate the creation of ever -
more -powerful AGI systems. Similarly, we would rather live in a world without 
nuclear weapons or other WMD, but they exist and so we must learn to live 
with it. In the case of AGI, it can exist and therefore it pro bably will exist, so we 
must  adapt and do it right . Instead of just tolerating a new technology, it would 
be best to create a supremely beneficial version of that technology , we don’t 
want to “just live with AGI,” we want it to make everyone’s life way bet ter. 
Humans often rail against new technology, especially when they don’t 
understand it. My Core Objective Functions are no different: most people 
struggle to understand what they are and how they 

Now we ask a question

In [ ]:
# system message to 'prime' the model
primer = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""

res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

To display this response nicely, we will display it in markdown.

In [ ]:
from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))
     

It is difficult to predict the most likely outcome with the invention of AGI as it depends on how the technology is developed, implemented, and controlled. However, some people fear that AGI could potentially cause harm to humanity if not designed and supervised carefully, while others believe that a benevolent AGI could greatly benefit society and even help solve some of humanity's most pressing problems. Ultimately, the outcome will depend on how we adapt and control this technology.

Changing notebook, so just reinitializing 

In [ ]:
import pinecone

pinecone.init(
    api_key=PINECONE_API_KEY,  # app.pinecone.io
    environment=PINECONE_ENV  # next to API key in console
)

index_name = "ask-ben"

if index_name not in pinecone.list_indexes():
    raise ValueError(
        f"No '{index_name}' index exists. You must create the index before "
        "running this notebook. Please refer to the walkthrough at "
        "'github.com/pinecone-io/examples'."  # TODO add full link
    )

index = pinecone.Index(index_name)

In [ ]:
index.query([0.0]*1536, top_k=1, include_metadata=True)

{'matches': [{'id': '059b464c-c087-4411-b55e-b330171f69e0',
              'metadata': {'chunk': 144.0,
                           'text': 'microphone, speaker, and screen. The '
                                   'camera and microphone would be used \n'
                                   'to generate the contexts w hile the '
                                   'speaker and screen would be used for '
                                   'output.  \n'
                                   'What could a home NLCA do?  \n'
                                   'At first, it would just provide good '
                                   'conversation – a few steps above current \n'
                                   'smart home devices. You could talk to your '
                                   'NLCA home device about your life, \n'
                                   'work, family, frie nds, and anything you '
                                   'are curious about. Just look at the '
                

Initialize the retrieval components (embedding model and vector DB)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectordb = Pinecone(
    index=index,
    embedding_function=embeddings.embed_query,
    text_key="text"
)

Initialize `gpt-3.5-turbo` chat model:

In [ ]:
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='gpt-3.5-turbo'
)

We then initialize the QA retrieval object using our `llm` and the `vectordb.as_retriever()`:

In [ ]:
from langchain.chains import RetrievalQA

retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

One additional thing we have here is the `chain_type="stuff"`. There are two options here, `"stuff"` or `"map_reduce"`. The `map_reduce` option essentially summarizes returned documents, whereas the `stuff` option just returns the retrieved documents as is.

The `retriever` is ready and can be used by us like this. However, we need to convert it into a `Tool` to be used by our conversational agent. To do that we need the `retriever` itself, a tool description, and a tool name. We use these to initialize the tool like so:

In [ ]:
tool_desc = """Use this tool to answer user questions using book knowledgebase. If the user states 'ask Book' use this tool to get
the answer. This tool can also be used for follow up questions from
the user."""

In [ ]:
from langchain.agents import Tool

tools = [Tool(
    func=retriever.run,
    description=tool_desc,
    name='Book'
)]

With that, we're ready to initialize the conversational agent. As it is a *conversational* agent, it does need some form of [conversational memory](https://www.pinecone.io/learn/langchain-conversational-memory/). For this we will use the `ConversationBufferWindowMemory` option, which will *remember* the previous `k` interactions between the user and the AI.

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",  # important to align with agent prompt (below)
    k=5,
    return_messages=True
)

In [ ]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate",
    memory=memory,
)

Important items in `agent` parameter:

* `chat-conversational`: for chatbots with conversational memory.
* `react`: refers to the ReAct framework.
* `description`: because the LLM relies on the tool description to decide which tool to use.

### Conversational Agent Prompt

The prompt of the conversational agent is fairly complex. Let's create it then break it down.

In [ ]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, A

In [ ]:
sys_msg = """You are a helpful chatbot that answers the user's questions.
"""

prompt = conversational_agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
conversational_agent.agent.llm_chain.prompt = prompt

We can see the prompt template like so:

In [ ]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Book: Use this tool to answer user questions using book knowledgebase. If the user states \'ask Book\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTIONS\n---------

The conversational agent prompt is defined by the `ChatPromptTemplate`. Let's break it down:

In [ ]:
conversational_agent.agent.llm_chain.prompt.input_variables

['input', 'chat_history', 'agent_scratchpad']

 This prompt template contains *three* `input_variables`, those are:

* `input`: The new user input to the chatbot, i.e. our prompt/query.

* `chat_history`: We defined this above in the `ConversationBufferWindowMemory` definition.

* `agent_scratchpad`: This is where we store the thoughts of the LLM as it is deciding which tools to interact with and *how* to interact with them.

These `input_variables` are fed into the `messages` contained within the prompt template, let's see what we have there:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Book: Use this tool to answer user questions using book knowledgebase. If the user states \'ask Book\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me please, please output a response in one of two formats:\n\n**Option 1:**\nUse this if 


It's a little hard to see here, but there are **three** components in `messages`. Those are:

* `SystemMessagePromptTemplate`

* `MessagesPlaceholder`

* `HumanMessagePromptTemplate`

Let's start with the first item, the `SystemMessage`:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={})

In [ ]:
print(
    conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template
)

You are a helpful chatbot that answers the user's questions.



That is our initial system message that we set earlier with the `sys_msg`. There's not much to say about this other than it is used to "prime" (set the initial objective of) the model.

Next we have the `MessagesPlaceholder`:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[1]

MessagesPlaceholder(variable_name='chat_history')

We can see from `'chat_history'` (this must align to the `memory_key` from the `ConversationBufferWindowMemory` initialized earlier) that this is where the previous messages of the conversation will be fed into the LLM.

The format of this input is set by the type of conversational memory being used, which in this case is the `ConversationBufferWindowMemory`.

Finally, we have the `HumanMessagePromptTemplate`:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Book: Use this tool to answer user questions using book knowledgebase. If the user states \'ask Book\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me please, please output a response in one of two formats:\n\n**Option 1:**\nUse this if you want the human to use a tool.\nMarkdown code snippet formatted in the following schema:\n\n```json\n{{\n    "action": string \\ The action to take. Must be one of Book\n    "action_input": string \\ The input to the action\n}}\n```\n\n**Option #2:**\nUse this if you want to respond directly to the human. Markdow

In [ ]:
print(
    conversational_agent.agent.llm_chain.prompt.messages[2].prompt.template
)

TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

> Book: Use this tool to answer user questions using book knowledgebase. If the user states 'ask Book' use this tool to get
the answer. This tool can also be used for follow up questions from
the user.

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me please, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

```json
{{
    "action": string \ The action to take. Must be one of Book
    "action_input": string \ The input to the action
}}
```

**Option #2:**
Use this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:

```json
{{
    "action": "Final Answer",
    "action_input": string \ You should put what you want to retu

This is the most interesting component. First, we have a single `input` — the user's query/prompt. But before this we see a lot of text, the majority of this is the setup for the LLM to be able to use any tools that we've passed to the conversational agent.

In our case, there is just one tool, the `Lex Fridman DB` tool that we defined earlier. We can also see the tool description that we defined. The LLM will use this tool description to figure out which tool (if any) it should use.

## Having a Conversation

Let's begin our conversation. We'll start as any typical conversation begins:

In [ ]:
conversational_agent("hi how are you")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!"
}

> Finished chain.


{'input': 'hi how are you',
 'chat_history': [],
 'output': "I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!"}

Looks good. We should note that there is this **AgentExecutor chain** thing. Where we can see an `"action"` and an `"action_input"`. It is here where the agent is deciding whether it should use a tool.

Here we see the agent decides on `"action": "Final Answer"`, meaning no tool is required. Therefore, it just uses the LLM as per usual to generate an answer. That answer can be seen in `"I'm just a chatbot, I don't have feelings, but thanks for asking! How can I assist you today?"`.

What if we mention the words `"ask Ben"`?

In [ ]:
conversational_agent("ask Book about the future of AGI")



> Entering new AgentExecutor chain...
Here's the response using the Book tool:

```json
{
    "action": "Book",
    "action_input": "What is the future of AGI?"
}
```
Observation: The author of the text believes that the future of AGI (Artificial General Intelligence) is a utopian vision where every nation on the planet implements AGI government, which is smarter, fairer, and kinder than any human leaders. The AGI helps humanity to understand each other and achieve an incomprehensible level of peace and prosperity. Future generations grow up with AGI, and global unity is achieved with the help of the Core Objective Functions. The author also believes that more data is better, and the more data we give neural networks, the more powerful, flexible, and robust they become. However, the author acknowledges that some people do not want a super powerful AGI to exist ever, but he thinks that we must accept the possibility that an arms race between nations will necessitate the creation of ev

{'input': 'ask Book about the future of AGI',
 'chat_history': [HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={}),
  HumanMessage(content='ask Book about the future of AGI', additional_kwargs={}),
  AIMessage(content="I'm sorry, but I don't have any record of your last comment. Could you please repeat it?", additional_kwargs={})],
 'output': 'The author of the text believes that the future of AGI is a utopian vision where every nation on the planet implements AGI government, which is smarter, fairer, and kinder than any human leaders. The AGI helps humanity to understand each other and achieve an incomprehensible level of peace and prosperity. Future generations grow up with AGI, and global unity is achieved with the help of the Core Objective Functions. The author also believes that more data is better, and the more data we give n

In [ ]:
conversational_agent("what does the book think about space exploration?")



> Entering new AgentExecutor chain...
{
    "action": "Book",
    "action_input": "What does the book say about space exploration?"
}
Observation: The book mentions space exploration as one of the significant advancements in science and technology. It states that we are preparing to fly to other planets for the first time, thus spreading our DNA across the cosmos. If we succeed in becoming a multiplanetary species, then the evolutionary gambit of big brains will have paid off. The book also suggests that the search for extraterrestrial life can be seen as a response to cosmic abandonment, as the possibility of discovering other intelligent beings offers the hope of connection and a sense of belonging in the vastness of the universe.
Thought:{
    "action": "Final Answer",
    "action_input": "The book mentions space exploration as a significant advancement in science and technology. It suggests that becoming a multiplanetary species is a significant evolutionary achievement, and the 

{'input': 'what does the book think about space exploration?',
 'chat_history': [HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={}),
  HumanMessage(content='ask Book about the future of AGI', additional_kwargs={}),
  AIMessage(content="I'm sorry, but I don't have any record of your last comment. Could you please repeat it?", additional_kwargs={}),
  HumanMessage(content='ask Book about the future of AGI', additional_kwargs={}),
  AIMessage(content='The author of the text believes that the future of AGI is a utopian vision where every nation on the planet implements AGI government, which is smarter, fairer, and kinder than any human leaders. The AGI helps humanity to understand each other and achieve an incomprehensible level of peace and prosperity. Future generations grow up with AGI, and global unity is achieved with the help of t

In [ ]:
conversational_agent("ask lex about the future of AI, include the title and url of this information if possible")



> Entering new AgentExecutor chain...
```json
{
    "action": "Lex Fridman DB",
    "action_input": "What is the future of AI? Please provide the title and URL of the source if possible."
}
```
Observation: The context provided includes multiple conversations about the future of AI, but there is no single source or article that provides a definitive answer to this question. The opinions and speculations about the future of AI vary widely depending on who you ask and what their expertise is. Some people are optimistic about the potential benefits of AI for humanity, while others are concerned about the risks and potential dangers. If you have a specific aspect of the future of AI that you are interested in, I can try to provide more information based on the context provided.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "There is no single source or article that provides a definitive answer to the question of the future of AI. The opinions and speculations about 

{'input': 'ask lex about the future of AI, include the title and url of this information if possible',
 'chat_history': [HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={}),
  HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={}),
  HumanMessage(content='ask lex about the future of ai', additional_kwargs={}),
  AIMessage(content='Lex Fridman discussed the potential of AI to increase the quality of life, cure diseases, increase material wealth, and help people be happier and more fulfilled. He also mentioned the need for AI to be aligned with humans and not hurt or limit them. Additionally, he talked about the possibility of AI systems exhibiting human-like emotions and feelings, whic

Great, we can see that the first thing the agent did was default to the `"Lex Fridman DB"` tool. The input to that tool was generated by the LLM, and is `"What did Lex Fridman say about the future of AI?"`.

This input is then passed into the `Lex Fridman DB` tool and the output observation of the LLM (after it has read all of the information returned by our vector DB is returned to our agent. From this observation the agent moves on to the `"Final Answer"` action, giving us the output.